<a href="https://colab.research.google.com/github/kkimtaejung/All_heuristic/blob/GAN/GAN_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 블로그 참고

https://m.blog.naver.com/euleekwon/221560040601

코드 안돌아감 버전 낮아서

## 패키지 불러오기

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## 데이터 불러오기 (MNIST)

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

## 데이터 확인하기

In [ ]:
print(mnist.train.images)
print("---------")
print(mnist.train.labels)
print("---------")
print(mnist.train.images, mnist.train.labels

## 파라미터 설정하기

In [ ]:
total_epochs = 100
batch_size = 100
learning_rate = 0.0002

## 신경망 레이어 Parameter 설정하기

In [ ]:
n_hidden = 256
n_input = 28 * 28
n_noise = 128 

## 신경망 모델 제작하기

In [ ]:
# Placeholder을 선언해준다.
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

### Generator 모델 구성

In [ ]:
# Generator 신경망에 사용하는 Variable를 선언한다.
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

# 생성기(G) 신경망을 구성한다(128 -> 256 -> 28*28)
def generator(noise_z):
    hidden = tf.nn.relu(
                    tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden, G_W2) + G_b2)

    return output

### Discriminator 모델 구성

In [ ]:
# Discriminator 신경망에 사용하는 Variable을 선언한다.
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

# 판별기(D) 신경망을 구성한다.(28*28 -> 256 -> 1)
def discriminator(inputs):
    hidden = tf.nn.relu(
                    tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(
                    tf.matmul(hidden, D_W2) + D_b2)

    return output

### 랜덤 노이즈 생성한 G,D 사용

In [ ]:
# 랜덤한 Noise를 만든다.
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

G = generator(Z) # 노이즈를 이용해 랜덤한 이미지를 생성한다.
D_gene = discriminator(G) # 노이즈를 이용해 생성한 이미지가 진짜 이미지인지 판별한 값을 구한다.
D_real = discriminator(X) # 진짜 이미지를 이용해 판별한 값을 구한다.

### loss, optimiser, learning rate 설정

In [ ]:
loss_D = -tf.reduce_mean(tf.log(D_real) + tf.log(1 - D_gene))
loss_G = -tf.reduce_mean(tf.log(D_gene)

D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

train_D = tf.train.AdamOptimizer(learning_rate).minimize(loss_D,
                                                         var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(loss_G,
                                                         var_list=G_var_list)

## 신경망 모델 학습하기

In [ ]:
sess = tf.Session() #Launch session.
sess.run(tf.global_variables_initializer()) #변수 초기화 진행.

total_batch = int(mnist.train.num_examples/batch_size) 
loss_val_D, loss_val_G = 0, 0 #Loss값 초기값 설정

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)

        # 판별기와 생성기 신경망을 각각 학습.
        _, loss_val_D = sess.run([train_D, loss_D],
                                 feed_dict={X: batch_xs, Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G],
                                 feed_dict={Z: noise})

    # Epoch 가 진행될때마다 D의 Loss 값과 G의 Loss 값을 출력한다.
    print('Epoch:', '%04d' % epoch,
          'D loss: {:.4}'.format(loss_val_D),
          'G loss: {:.4}'.format(loss_val_G))

    # 주기적으로 이미지를 생성하여 저장
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})

        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))

        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))

        #result 라는 폴더를 미리 만들어서 사용하였다.
        plt.savefig('./result/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
        plt.close(fig)

print('최적화 완료!')